In [1]:
import pickle
import numpy as np

In [2]:
file = "../DeepMoji-master/data/PsychExp/raw.pickle"

In [3]:
data = pickle.load(open(file,'rb'),encoding='latin1')

In [4]:
label = data['info']

texts = data['texts']

val_index = data['val_ind']

test_index = data['test_ind']

train_index = data['train_ind']

In [5]:
def glove_vec(path):
    import numpy as np
    with open(path,'r',encoding='utf-8') as f:
        words = set()
        word_vec = {}
        for line in f:
            line = line.strip().split()
            key = line[0]
            words.add(key)
            val = np.array(line[1:],dtype=np.float64)
            word_vec[key] = val
            
        # Dict (Vocab) --> set
        word2idx = {}
        idx2word = {}
        
        for i,w in enumerate(sorted(words)):
            word2idx[w] = i
            idx2word[i] = w
            
    return word_vec,word2idx,idx2word

In [6]:
word_vec,word2idx,idx2word = glove_vec('../glove.6B/glove.6B.50d.txt')
vocab_size = len(word_vec)

In [7]:
from nltk import word_tokenize

In [8]:
def get_XY_data(Index):
    X = np.zeros((len(Index) , 128 ))
    Y = np.zeros((len(Index) , 7))
    
    zer = np.zeros((50 ,))
    for i ,index in enumerate(Index):
        line = word_tokenize(texts[index].lower())
        
        #print(index)
        Y[i] = label[index]['label']
        for j , word  in  enumerate(line[:128]):
             X[i , j] =  word2idx.get(word , 0)
    
    return X ,Y

In [9]:
X_train , Y_train  = get_XY_data(train_index)
print(X_train.shape , Y_train.shape)

X_test , Y_test  = get_XY_data(test_index)
print(X_test.shape , Y_test.shape)

X_val , Y_val  = get_XY_data(val_index)
print(X_val.shape , Y_val.shape)


(900, 128) (900, 7)
(6480, 128) (6480, 7)
(100, 128) (100, 7)


In [10]:
from keras.layers import Embedding , Flatten ,Dense , Bidirectional ,LSTM
from keras.models import Sequential
from keras.layers import BatchNormalization

In [27]:
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=128))
model.add(Bidirectional(LSTM(128 ,  return_sequences = True )))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(7, activation='softmax'))

In [28]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 128, 50)           20000000  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128, 256)          183296    
_________________________________________________________________
batch_normalization_1 (Batch (None, 128, 256)          1024      
_________________________________________________________________
flatten_1 (Flatten)          (None, 32768)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 7)                 229383    
Total params: 20,413,703
Trainable params: 20,413,191
Non-trainable params: 512
_________________________________________________________________
None


In [ ]:
history = model.fit(X_train , Y_train , batch_size = 10 , epochs=5 , validation_data=(X_val,Y_val))

Epoch 1/5
90/90 [==============================] - 25s 281ms/step - loss: 2.0546 - accuracy: 0.1878 - val_loss: 1.9249 - val_accuracy: 0.2400
Epoch 2/5
90/90 [==============================] - 26s 289ms/step - loss: 0.9492 - accuracy: 0.6889 - val_loss: 1.8809 - val_accuracy: 0.3000
Epoch 3/5
90/90 [==============================] - 26s 287ms/step - loss: 0.2278 - accuracy: 0.9400 - val_loss: 1.8200 - val_accuracy: 0.2300
Epoch 4/5
31/90 [=========>....................] - ETA: 16s - loss: 0.0343 - accuracy: 0.9968

In [26]:
model.evaluate(X_test,Y_test)

203/203 [==============================] - 9s 47ms/step - loss: 9.1706 - accuracy: 0.2721


[9.170609474182129, 0.2720679044723511]